In [ ]:
import pandas as pd
import re
import hanja

In [ ]:
data = pd.read_excel("/content/drive/MyDrive/poem_crawled_data.xlsx")
print(len(data))
data.head()

In [ ]:
#영어 들어간 content 지우기
data['drop'] = data['content'].apply(lambda x : 1 if  re.compile(r'[a-zA-Z]+').search(x) else 0)
data = data.drop(data[data['drop']==1].index)

In [ ]:
#괄호 + 괄호 안 지우기
data['content'] = data['content'].apply(lambda x : re.sub(r"\([^)]*\)","",x))

In [ ]:
#줄바꿈 바꾸기
def func_space(text):
  text = re.sub(r"\/+","/",text)
  text = re.sub(r"[ (\xa0)|(\t)]+"," ",text)
  text = re.sub(r"[(\n)]+","\n",text)
  return text

data['content'] = data['content'].apply(lambda x : func_space(x))

In [ ]:
def func_character(text):
  #한자 바꾸기
  hj = list(set(re.findall(r"([一-鿕]|[㐀-䶵]|[豈-龎])+", text)))
  hg = [hanja.translate(h,'substitution') for h in hj]
  for i in range(len(hg)):
    text = text.replace(hj[i], hg[i])
  #특수문자 지우기
  text = re.sub(r"[^가-힇0-9‘“.,!? ]+", "", text)
  return text

data['content'] = data['content'].apply(lambda x : func_character(x))

In [ ]:
data['drop'] = data['content'].apply(lambda x : 1 if len(x)<10 else 0)
data = data.drop(data[data['drop']==1].index)

In [ ]:
len(data)
data = data[['title', 'content', 'img_url']]
data.to_excel("preprocess_data.xlsx")